In [1]:
import scipy.sparse as sps
import numpy as np
import os 
import sys 

sys.path.insert(0,os.pardir)
try:
    import sparse_dot_mkl
    dot_product = sparse_dot_mkl.dot_product_mkl
except ImportError:
    def dot_product(x, y, dense=False, **kwargs):
        z = x @ y
        return z.A if dense and sps.issparse(z) else z 

In [21]:
np.random.seed(0)
X = np.random.randn(500, 30000)
X[X < 1.5] = 0
X = sps.csr_matrix(X)
print(f'X sparsity: {100 * (1 - X.count_nonzero() / np.prod(X.shape)):5.2f} %')

X sparsity: 93.32 %


In [22]:
X @ X.T

<500x500 sparse matrix of type '<class 'numpy.float64'>'
	with 250000 stored elements in Compressed Sparse Row format>

In [23]:
expected_result = (X @ X.T).toarray()
expected_result_tril = expected_result.copy()
expected_result_tril[np.tril_indices(expected_result.shape[0], k=-1)] = 0

In [24]:
mkl_result1 = sparse_dot_mkl.dot_product_mkl(X, X.T)
np.allclose(mkl_result1.toarray(), expected_result)

True

In [25]:
mkl_result2 = sparse_dot_mkl.gram_matrix_mkl(X, transpose=True)
np.allclose(mkl_result2.toarray(), expected_result_tril)

True

In [27]:
print("Scipy Matrix Multiplication Product:")
%timeit X @ X.T

print("MKL Matrix Multiplication Product:")
%timeit sparse_dot_mkl.dot_product_mkl(X, X.T)

print("MKL Gram Matrix Product:")
%timeit sparse_dot_mkl.dot_product_transpose_mkl(X, transpose=True)

Scipy Matrix Multiplication Product:
205 ms ± 17.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
MKL Matrix Multiplication Product:
63 ms ± 4.54 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
MKL Gram Matrix Product:
72.6 ms ± 8.07 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
np.random.seed(0)
X = np.random.randn(500, 5000)
X[X < 0.8] = 0
X = sps.csr_matrix(X)

Y = np.random.randn(500, 5000)
Y[Y < 0.8] = 0
Y = sps.csr_matrix(Y)


In [11]:
print("Scipy Matrix Multiplication Product:")
%timeit X @ Y.T

print("MKL Matrix Multiplication Product:")
%timeit sparse_dot_mkl.dot_product_mkl(X, Y.T)


Scipy Matrix Multiplication Product:
130 ms ± 4.69 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
MKL Matrix Multiplication Product:
41.5 ms ± 1.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
